In [4]:
import torch

torch.__version__


'2.5.1'

PyTorch 张量是用于存储类似数组结构的数据容器。标量是 0 维张量（例如，一个简单的数字），向量是 1 维张量，而矩阵是 2 维张量。对于更高维度的张量没有特定的术语，所以我们通常将 3 维张量称为 3D 张量，以此类推。

我们可以使用 torch.tensor 函数创建 PyTorch 的 Tensor 类的对象，如下所示：

In [9]:
# Listing A.1 Creating PyTorch tensors

tensor0d = torch.tensor(1)                                    #A
tensor1d = torch.tensor([1, 2, 3])                            #B
tensor2d = torch.tensor([[1, 2], [3, 4]])                     #C
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]]) #D



#A 从 Python 整数创建一个 0 维张量（标量）
#B 从 Python 列表创建一个 1 维张量（向量）
#C 从嵌套的 Python 列表创建一个 2 维张量
#D 从嵌套的 Python 列表创建一个 3 维张量

在Tensor的数据类型中，主流的是什么，为什么不选择浮点数

In [8]:
print(tensor0d.dtype)
print(tensor1d.dtype)
print(tensor2d.dtype)
print(tensor3d.dtype)

floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

torch.int64
torch.int64
torch.int64
torch.int64
torch.float32


这种选择主要是基于精度和计算效率之间的平衡。对于大多数深度学习任务来说，32 位浮点数提供了足够的精度，同时比 64 位浮点数消耗更少的内存和计算资源。此外，GPU 架构针对 32 位计算进行了优化，使用这种数据类型可以显著加快模型训练和推理的速度。

In [12]:
float_tensor_1d = tensor1d.to(torch.float32)
print(float_tensor_1d.dtype)

print(tensor0d)
print(tensor0d.shape)
print("--------")
print(tensor1d)
print(tensor1d.shape)
print("--------")
print(float_tensor_1d)
print(float_tensor_1d.shape)
print("--------")
print(tensor2d)
print(tensor2d.shape)
print("--------")
print(tensor3d)
print(tensor3d.shape)
print("--------")

torch.float32
tensor(1)
torch.Size([])
--------
tensor([1, 2, 3])
torch.Size([3])
--------
tensor([1., 2., 3.])
torch.Size([3])
--------
tensor([[1, 2],
        [3, 4]])
torch.Size([2, 2])
--------
tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
torch.Size([2, 2, 2])
--------


一个具体的例子展示简单逻辑回归分类器的前向传播

In [13]:
# Listing A.2 A logistic regression forward pass

import torch.nn.functional as F #A

y = torch.tensor([1.0])         #B
x1 = torch.tensor([1.1])        #C
w1 = torch.tensor([2.2])        #D
b = torch.tensor([0.0])         #E
z = x1 * w1 + b                 #F
a = torch.sigmoid(z)            #G

loss = F.binary_cross_entropy(a, y)
print(loss)

#A 这是 PyTorch 中常见的导入约定，用于避免代码行过长
#B 真实标签
#C 输入特征
#D 权重参数
#E 偏置单元
#F 网络输入
#G 激活与输出

tensor(0.0852)


 __init__ 构造函数中定义网络层，并在 forward 方法中指定它们如何交互。

In [14]:
from torch.autograd import grad

# Listing A.4 A multilayer perceptron with two hidden layers

class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):          #A
        super().__init__()
        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),              #B
            torch.nn.ReLU(),                              #C

            # 2nd hidden layer
            torch.nn.Linear(30, 20),                      #D
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        return logits                                     #E
      

#A 将输入和输出的数量编码为变量很有用，这样可以为具有不同特征和类别数量的数据集重用相同的代码。
#B Linear 层将输入和输出节点的数量作为参数。
#C 非线性激活函数放置在隐藏层之间。
#D 一个隐藏层的输出节点数必须与下一个隐藏层的输入节点数相匹配。
#E 最后一层的输出被称为 logits。

In [15]:
model = NeuralNetwork(50,3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


请注意，在实现 NeuralNetwork 类时，我们使用了 Sequential 类。使用 Sequential 不是必需的，但如果我们有一系列想要按特定顺序执行的层（就像这里的情况一样），它可以使我们的工作更轻松。这样，在 __init__ 构造函数中实例化 self.layers = Sequential(...) 之后，我们只需要调用 self.layers，而无需在 NeuralNetwork 的 forward 方法中单独调用每个层。

请注意，每个 requires_grad=True 的参数都被认为是可训练参数，并且将在训练期间更新（更多内容请参见 2.7 节“一个典型的训练循环”）。
对于我们上面定义的具有两个隐藏层的神经网络模型，这些可训练参数包含在 torch.nn.Linear 层中。一个线性层将输入与权重矩阵相乘，并加上一个偏置向量。这有时也被称为前馈层或全连接层。
根据我们上面执行的 print(model) 调用，我们可以看到第一个 Linear 层位于 layers 属性的索引位置 0。

In [17]:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)


Parameter containing:
tensor([[-0.0250, -0.0269,  0.0146,  ...,  0.0749, -0.0085, -0.0170],
        [-0.1405, -0.0268,  0.1258,  ..., -0.0536, -0.0714,  0.0032],
        [-0.0794,  0.0754, -0.0627,  ...,  0.1195,  0.1399, -0.1105],
        ...,
        [ 0.1389,  0.0277,  0.0631,  ..., -0.0038, -0.1367,  0.0272],
        [-0.0591, -0.1273,  0.0071,  ...,  0.0970,  0.0499,  0.0907],
        [ 0.0583, -0.1370, -0.1158,  ...,  0.0260, -0.0522, -0.0721]],
       requires_grad=True)
torch.Size([30, 50])


这些权重的初始化是随机的，因此需要使用种子来固定这些随机化的权重

In [18]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [19]:
# Listing A.5 Creating a small toy dataset

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [20]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x,one_y
    
    def __len__(self):
        return self.labels.shape[0]
    
train_dataset = ToyDataset(X_train,y_train)
test_dataset = ToyDataset(X_test,y_test)

In [21]:
print(len(test_dataset))

2


现在我们已经为我们的数据集定义了一个 PyTorch Dataset 类，接着可以使用 PyTorch 的 DataLoader 类来从中采样数据，如下面的代码清单所示：

In [22]:
# Listing A.7 Instantiating data loaders

from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,       #A
    batch_size=2,
    shuffle=True,           #B
    num_workers=0           #C
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=2,
    shuffle=False,          #D
    num_workers=0
)


#A 之前创建的 ToyDataset 实例作为数据加载器的输入。
#B 是否打乱数据
#C 后台进程的数量
#D 没有必要打乱测试数据
for idx, (x, y) in enumerate(train_loader):
		print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


为什么要shuffle：因为如果不打乱顺序，模型很可能只记住了这些数据之间的顺序关系，而无法真正理解内容。